In [1]:
#dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pymongo
import time
import numpy as np
import requests
import pandas as pd
from sqlalchemy import create_engine
from random import random
import random

In [2]:
browser = Browser('chrome', headless=True)

In [3]:
def get_soup(url):
    browser.visit(url)
    html = browser.html    
    soup = bs(html, 'html.parser')
    return soup

In [4]:
def get_author(url):
    result = {}
    response = requests.get(url)
    soup =  BeautifulSoup(response.text, 'lxml')    
    result['born'] = soup.find('span' , class_ = 'author-born-date').text.strip()
    result['name'] = soup.h3.text.strip()
    result['description'] = soup.find('div' , class_="author-description").text.strip()
    return result


In [5]:
# #scraping top ten tags. 
# #flask /top10tags
# soup = get_soup('http://quotes.toscrape.com/')
# sidebar = soup.find_all('span', class_='tag-item')
# item = sidebar[0]
# tags = []

# for item in sidebar:
#     tag = item.a.getText()
#     tags.append(tag)
#     #print(tag)

In [7]:
#quotes, authors, tags 
#looping through EACH page
quote_all = []
pages = np.arange(1,11,1)#start page 1, stop page 10, step 1 
quote_id = 0
for page in pages:
    page = requests.get('http://quotes.toscrape.com/page/' +str(page)+ '/')
    soup = bs(page.text,'lxml') 
    #quotes = soup.find_all('span', class_='text')
    quotes = soup.find_all('div', class_='quote')
    authors = soup.find_all('small', class_='author')
    tags = soup.find_all('div', class_="tags")
    i = 1
    for i in range(0,len(quotes)):
        q = quotes[i].text
        a = authors[i].text
        link = quotes[i].a['href']
        tags = soup.find_all('div', class_='tags')
        #quotetags = tags[i].find_all('a',class_='tag')
        t = [ ] 
        for tags in quotes[i].find_all('a' , class_='tag'):
            
            taggers = tags.text
            t.append(taggers)

        author_url = 'http://quotes.toscrape.com' + link 
        author_info = requests.get('http://quotes.toscrape.com' + link)
        soup = bs(author_info.text,'html.parser')
        author_born = soup.find_all('span', class_='author-born-date')
        author_description = soup.find_all('div', class_='author-description')
        quote_id = quote_id + 1
        data = {
           'quote_id': quote_id,
           'quote': q,
           'author_name': a,
           'tags' : t,
            'dob': author_born[0].text,
            'description': author_description[0].text                
           }
        quote_all.append(data)
        
    else:
        continue



browser.quit()

In [84]:
quote_all_transformed = pd.DataFrame(quote_all)
quote_all_transformed['quote_text'] = quote_all_transformed['quote'].str.split('\n').str[1]
quote_all_transformed['description_text'] = quote_all_transformed['description'].str.split('\n').str[1]

In [85]:
del quote_all_transformed['quote']

In [86]:
del quote_all_transformed['description']

In [91]:
quote_all_transformed

,quote_id,author_name,tags,dob,_id,quote_text,description_text
0,1,Albert Einstein,"[change, deep-thoughts, thinking, world]","March 14, 1879",5fcaa99e54d46f7c56aa8fd7,“The world as we have created it is a process ...,"In 1879, Albert Einstein was born in U..."
1,2,J.K. Rowling,"[abilities, choices]","July 31, 1965",5fcaa99e54d46f7c56aa8fd8,"“It is our choices, Harry, that show what we t...",See also: Robert GalbraithAlthough she...
2,3,Albert Einstein,"[inspirational, life, live, miracle, miracles]","March 14, 1879",5fcaa99e54d46f7c56aa8fd9,“There are only two ways to live your life. On...,"In 1879, Albert Einstein was born in U..."
3,4,Jane Austen,"[aliteracy, books, classic, humor]","December 16, 1775",5fcaa99e54d46f7c56aa8fda,"“The person, be it gentleman or lady, who has ...",Jane Austen was an English novelist wh...
4,5,Marilyn Monroe,"[be-yourself, inspirational]","June 01, 1926",5fcaa99e54d46f7c56aa8fdb,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe (born Norma Jeane Morte...
...,...,...,...,...,...,...,...
95,96,Harper Lee,[better-life-empathy],"April 28, 1926",5fcaa99e54d46f7c56aa9036,“You never really understand a person until yo...,"Harper Lee, known as Nelle, was born i..."
96,97,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...","November 29, 1918",5fcaa99e54d46f7c56aa9037,“You have to write the book that wants to be w...,Madeleine L'Engle was an American writ...
97,98,Mark Twain,[truth],"November 30, 1835",5fcaa99e54d46f7c56aa9038,“Never tell the truth to people who are not wo...,"Samuel Langhorne Clemens, better known..."
98,99,Dr. Seuss,[inspirational],"March 02, 1904",5fcaa99e54d46f7c56aa9039,"“A person's a person, no matter how small.”",Theodor Seuss Geisel was born 2 March ...


In [88]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [90]:
#Load Quotes data
db = client.quote_etl
quote = db.author_quote
quote.insert_many(quote_all_transformed)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
#Execute postgres_schema.sql in postgres client to create the tables

In [11]:
#Define postgres connection
conn = f"postgres:postgres@localhost:5432/quotes_db"
engine = create_engine(f'postgresql://{conn}')

In [12]:
engine.table_names()

['author', 'quotes', 'tags']

In [18]:
quote_all_df = pd.DataFrame(quote_all)
quotes_df = quote_all_df[["quote_id","author_name","quote"]].copy()
author_df = quote_all_df[["author_name","dob","description"]].copy()
#tags_df = quote_all_df[["quote_id","tags"]]

In [15]:
#Load tags data
tags_table = []
quotes_table = []
author_table = []
for rows in db.author_quote.find({}):
    for tag in rows['tags']:
        tags_table.append({'quote_id':rows['quote_id'] , 'tag' : tag})
    quotes_table.append('quote_id':rows['quote_id'], '')
        
tags_df = pd.DataFrame(tags_table)
tags_df.to_sql(name='tags', con=engine, if_exists='append', index=False)

In [39]:
#Load author data

author_df.to_sql(name='author', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "author_pkey"
DETAIL:  Key (author_name)=(Albert Einstein) already exists.

[SQL: INSERT INTO author (author_name, dob, description) VALUES (%(author_name)s, %(dob)s, %(description)s)]
[parameters: ({'author_name': 'Albert Einstein', 'dob': 'March 14, 1879', 'description': '\n        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining th ... (3554 characters truncated) ... have made the word "Einstein" synonymous with genius.More: http://en.wikipedia.org/wiki/Albert_E...http://www.nobelprize.org/nobel_prize...    \n    '}, {'author_name': 'J.K. Rowling', 'dob': 'July 31, 1965', 'description': '\n        See also: Robert GalbraithAlthough she writes under the pen name J.K. Rowling, pronounced like rolling, her name when her first Harry Potte ... (3510 characters truncated) ... d at English." Sean Harris, her

In [31]:
author_df

,author_name,DOB,Description
0,Albert Einstein,"March 14, 1879","\n In 1879, Albert Einstein was born in..."
1,J.K. Rowling,"July 31, 1965",\n See also: Robert GalbraithAlthough s...
2,Albert Einstein,"March 14, 1879","\n In 1879, Albert Einstein was born in..."
3,Jane Austen,"December 16, 1775",\n Jane Austen was an English novelist ...
4,Marilyn Monroe,"June 01, 1926",\n Marilyn Monroe (born Norma Jeane Mor...
...,...,...,...
95,Harper Lee,"April 28, 1926","\n Harper Lee, known as Nelle, was born..."
96,Madeleine L'Engle,"November 29, 1918",\n Madeleine L'Engle was an American wr...
97,Mark Twain,"November 30, 1835","\n Samuel Langhorne Clemens, better kno..."
98,Dr. Seuss,"March 02, 1904",\n Theodor Seuss Geisel was born 2 Marc...


In [22]:
#Load quotes data
quotes_df['quote_text'] = quotes_df["quote"].str.split('\n').str[1]
del quotes_df['quote']
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False)

In [51]:

#quotes_df